In [5]:
from urllib.request import urlopen 
from urllib.parse import urlparse 
from bs4 import BeautifulSoup 
import re
import datetime
import random

pages = set()
random.seed(datetime.datetime.now())

#Retrieves a list of all Internal links found on a page
def getInternalLinks(bs, includeUrl):
    includeUrl = '{}://{}'.format(urlparse(includeUrl).scheme,urlparse(includeUrl).netloc)
    internalLinks = []
    #Finds all links that begin with a "/" 
    for link in bs.find_all('a',href=re.compile('^(/|.*'+includeUrl+')')):
        #print('link = ' + link)
        if link.attrs['href'] is not None:
            if link.attrs['href'] not in internalLinks:
                if(link.attrs['href'].startswith('/')):
                    internalLinks.append(includeUrl+link.attrs['href'])
                else:
                    internalLinks.append(link.attrs['href'])
    return internalLinks

#Retrieves a list of all external links found on a page
def getExternalLinks(bs, excludeUrl):
    externalLinks = []
    #Finds all links that start with "http" that do not contain the current URL
    for link in bs.find_all('a',href=re.compile('^(http|www)((?!'+excludeUrl+').)*$')):
        if link.attrs['href'] is not None:
            if link.attrs['href'] not in externalLinks:
                externalLinks.append(link.attrs['href'])
    return externalLinks

def getRandomExternalLink(startingPage):
    html = urlopen(startingPage)
    bs = BeautifulSoup(html, 'html.parser')
#    print('urlparse(startingPage).netloc = ' + urlparse(startingPage).netloc)
    externalLinks = getExternalLinks(bs,urlparse(startingPage).netloc) #<scheme>://<netloc>/<path>;<params>?<query>#<fragment>
    if len(externalLinks) == 0:
        print('No external links, looking around the site for one')
        domain = '{}://{}'.format(urlparse(startingPage).scheme,urlparse(startingPage).netloc)
        internalLinks = getInternalLinks(bs, domain)
        return getRandomExternalLink(internalLinks[random.randint(0,len(internalLinks)-1)])
    else:
        return externalLinks[random.randint(0, len(externalLinks)-1)]
    
def followExternalOnly(startingSite):
    externalLink = getRandomExternalLink(startingSite)
    print('Random external link is: {}'.format(externalLink))
    followExternalOnly(externalLink)
    
followExternalOnly('http://oreilly.com')

Random external link is: https://twitter.com/oreillymedia
Random external link is: https://t.co/qCxaBK1J6d
Random external link is: https://www.computerweekly.com/microscope/news/252461255/Channel-roundup-Whos-gone-where
Random external link is: https://www.techtarget.com/solutions/
Random external link is: https://www.linkedin.com/company/techtarget


HTTPError: HTTP Error 999: Request denied